# Calculate FCM Statistics (for Singles)
*Written by Viviane Clay*
## Preparation
### Import Dependencies

In [14]:
import os
import numpy as np;
import matplotlib.pyplot as plt
from scipy import misc
import matplotlib as mpl
from itertools import combinations 
import pandas as pd
from sklearn import svm

import warnings
warnings.filterwarnings('ignore')

### Load Data
Get activations using the 'GetNetworkActivations.ipynb' script. Adapt paths as nescessary.

In [15]:
path = '../../Results/TowerTraining/BalancedTestSet/'
figurePath = '../../Results/TowerTraining/Figures/AgentRewardComparisonsAdaTH/ReviewFigures/SingleFCM/'

encExt = np.load(path + 'encExt.npy')
encExtInt = np.load(path + 'encExtInt.npy')
encInt = np.load(path + 'encInt.npy')
encAE = np.load(path + 'encAE.npy')
encBYOL = np.load(path + 'encBYOL.npy')
encC = np.load(path + 'encC.npy')
ClassOut = np.load(path + 'outClass.npy')
y = np.load(path + 'labels.npy')
obs = np.load(path + 'obs.npy')
# generate random activations
# randAct = np.random.randint(0,2,encC.shape)
randAct = np.load(path + 'encRand.npy')

### Apply Adaptive Threshold to Activations

In [16]:
c1ActTH = np.mean(encInt, axis = 0)
c2ActTH = np.mean(encExt, axis = 0)
c3ActTH = np.mean(encExtInt, axis = 0)
AEActTH = np.mean(encAE, axis = 0)
BYOLActTH = np.mean(encBYOL, axis = 0)
CActTH = np.mean(encC, axis = 0)
RActTH = np.mean(randAct, axis = 0)

encInt = np.array(np.abs(encInt) > np.abs(c1ActTH),dtype=int)
encExt = np.array(np.abs(encExt) > np.abs(c2ActTH),dtype=int)
encExtInt = np.array(np.abs(encExtInt) > np.abs(c3ActTH),dtype=int)
encAE = np.array(np.abs(encAE) > np.abs(AEActTH),dtype=int)
encBYOL = np.array(np.abs(encBYOL) > np.abs(BYOLActTH),dtype=int)
encC = np.array(np.abs(encC) > np.abs(CActTH),dtype=int)
encR = np.array(np.abs(randAct) > np.abs(RActTH),dtype=int)

### Helper Functions

In [17]:
def getEdges(encodings):
    TH = 0.5
    comb = combinations(np.linspace(0,encodings.shape[1]-1,encodings.shape[1]), 2)
    c = list(comb)
    edges = np.zeros(len(c))  

    for i in range(encodings.shape[0]):
        for n in range(len(c)):
            a = int(c[n][0])
            b = int(c[n][1])
            if (encodings[i][a]>TH) & (encodings[i][b]>TH):
                edges[n] = edges[n]+1
    return edges,c

def learnConcept(examples, name, pattern_complexity,normalize = False):
    e,c = getEdges(examples)
    strongest = e.argsort()[-pattern_complexity:][::-1]
    strongest_combs = []
    strengths = []
    #print('Combinations:      Strength:')
    for s in strongest:
        #print(str(c[s]) + '        ' + str(int(e[s])))
        strongest_combs.append(c[s])
        strengths.append(int(e[s]))
    if normalize:
        strengths = strengths/np.sum(strengths)
    else:
        strengths = np.array(strengths)/examples.shape[0]
    return strongest_combs, strengths


def getActsLogic(neuronCombs, strengths, data, class_TH):
    patternOn = np.zeros(data.shape[0])
    for i,n in enumerate(neuronCombs):
        n1_act = (data[:,int(n[0])] > 0.5)
        n2_act = (data[:,int(n[1])] > 0.5)
        n_act = np.array(n1_act & n2_act, dtype=int) * strengths[i]
        patternOn = patternOn + n_act
    patternOn = (patternOn >= class_TH)
    return patternOn

def plotPatternActsLogic(patternOn, labels):
    
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.plot(patternOn)
    plt.plot(labels/2)
    plt.xlim([0,labels.shape[0]])
    plt.ylim([0,1.1])
    plt.legend(['Pattern On', 'Label Present'])
    plt.yticks([])
    plt.xlabel('Frames')
    
    plt.subplot(1,2,2)
    c = plt.hist(patternOn + labels/2, bins = [0,0.5,1,1.5,2])
    plt.xticks([0.25,0.75,1.25,1.75],['TN', 'FN', 'FP','TP'])
    correct = np.round((c[0][0] + c[0][3])/labels.shape[0]*100,2)
    precision = np.round(c[0][3]/(c[0][3]+c[0][2])*100,2)# If L is predicted it is correct X% of time
    recall = np.round(c[0][3]/(c[0][3]+c[0][1])*100,2)# Correctly identifies X% of L
    plt.title('Correct: ' + str(correct) + '% Precision: ' + str(precision) + '% Recall: ' + str(recall) + '%')
    plt.show()
    
def plotCPR(combs, weights, enc, labels, THs):
    cpr = np.zeros((THs.shape[0],3))
    for i,TH in enumerate(THs):
        patternOn = getActsLogic(combs, weights, enc, class_TH=TH)
        c = plt.hist(patternOn + labels/2, bins = [0,0.5,1,1.5,2])
        
        correct = np.round((c[0][0] + c[0][3])/labels.shape[0]*100,2)
        precision = np.round(c[0][3]/(c[0][3]+c[0][2])*100,2)# If L is predicted it is correct X% of time
        recall = np.round(c[0][3]/(c[0][3]+c[0][1])*100,2)# Correctly identifies X% of L
        
        cpr[i] = np.array([correct,precision,recall])
    plt.clf()
    plt.figure(figsize=(15,5))
    plt.plot(cpr)
    plt.legend(['% Correct', 'Precision', 'Recall'])
    plt.xticks(np.linspace(0,THs.shape[0]-1,THs.shape[0]),np.round(THs,2))
    plt.xlim([0,THs.shape[0]-1])
    plt.ylim([0,100])
    plt.xlabel('Threshold')
    plt.ylabel('Performance [Accuracy in %]')
    plt.show()
    
def getCPR(combs, weights, enc, labels, THs,pattern):
    cpr = np.zeros((THs.shape[0],5))
    for i,TH in enumerate(THs):
        if pattern=='single':
            patternOn = getSingleActsLogic(combs, weights, enc, class_TH=TH)
        elif pattern=='double':
            patternOn = getActsLogic(combs, weights, enc, class_TH=TH)
        elif pattern=='triple':
            patternOn = getTripletsActsLogic(combs, weights, enc, class_TH=TH)
        else:
            print('unknown pattern identifier.')
            break
        c = plt.hist(patternOn + labels/2, bins = [0,0.5,1,1.5,2])
        
        correct = np.round((c[0][0] + c[0][3])/labels.shape[0]*100,2)
        precision = np.round(c[0][3]/(c[0][3]+c[0][2])*100,2)# If L is predicted it is correct X% of time
        recall = np.round(c[0][3]/(c[0][3]+c[0][1])*100,2)# Correctly identifies X% of L
        f1score = np.round(2*(precision*recall/(precision+recall)),2)
        
        cpr[i] = np.array([correct,precision,recall,correct+precision+recall,f1score])
    plt.close()
    return np.nan_to_num(cpr)

def getConceptSubset(AllConceptExamples, numExp):
    subset = np.random.randint(0,AllConceptExamples.shape[0],numExp)
    trainExp = AllConceptExamples[subset]
    mask = np.ones(AllConceptExamples.shape, bool)
    mask[subset] = False
    testExp = AllConceptExamples[mask]
    return trainExp,  testExp

def getFP_FN(o,l):
    w = np.where(l!=o)
    FN = np.sum((o[w]==0)&(l[w]==1))/len(w[0])*100
    FP = np.sum((o[w]>0)&(l[w]==0))/len(w[0])*100
    return np.round(FP,3),np.round(FN,3)

def getAccPR(o,l):
    w = np.where(l!=o)
    trueP = np.sum((o==l)&(o==1))
    trueN = np.sum((o==l)&(o==0))
    falseP = np.sum((o[w]>0)&(l[w]==0))
    falseN = np.sum((o[w]==0)&(l[w]==1))
    Acc = (trueP+trueN)/len(o)*100
    Precision = trueP/(trueP+falseP)*100
    Recall = trueP/(trueP+falseN)*100
    return np.round(Acc,3),np.round(Precision,3),np.round(Recall,3),[trueP,trueN,falseP,falseN]

def printStats(out, labels):
    print('Accuracy: '+str(np.mean(out == labels)*100)+'%')
    print('Everything correct in '+str(np.sum(np.mean(out == labels,axis=1)==1.0)/len(labels)*100)+'%')
    print('Correct Door in '+str(np.round(np.mean(np.array(labels)[:,0]==out[:,0])*100,3))+'%')

    AccND,PND,RND,NDA = getAccPR((out[:,0]==0).astype(int),(labels[:,0]==0).astype(int))
    print('     No Door - Acc: '+str(AccND)+'% Precision: '+str(PND)+'% Recall: '+str(RND)+'%  |  TP,TN,FP,FN: '+str(NDA))

    AccLD,PLD,RLD,LDA = getAccPR((out[:,0]==1).astype(int),(labels[:,0]==1).astype(int))
    print('     Level Door - Acc: '+str(AccLD)+'% Precision: '+str(PLD)+'% Recall: '+str(RLD)+'%  |  TP,TN,FP,FN: '+str(LDA))

    AccGD,PGD,RGD,GDA = getAccPR((out[:,0]==2).astype(int),(labels[:,0]==2).astype(int))
    print('     Green Door - Acc: '+str(AccGD)+'% Precision: '+str(PGD)+'% Recall: '+str(RGD)+'%  |  TP,TN,FP,FN: '+str(GDA))

    AccKD,PKD,RKD,KDA = getAccPR((out[:,0]==3).astype(int),(labels[:,0]==3).astype(int))
    print('     Key Door - Acc: '+str(AccKD)+'% Precision: '+str(PKD)+'% Recall: '+str(RKD)+'%  |  TP,TN,FP,FN: '+str(KDA))

    AccOD,POD,ROD,ODA = getAccPR((out[:,0]==4).astype(int),(labels[:,0]==4).astype(int))
    print('     Other Door - Acc: '+str(AccOD)+'% Precision: '+str(POD)+'% Recall: '+str(ROD)+'%  |  TP,TN,FP,FN: '+str(ODA))

    AccK,PK,RK,KA = getAccPR(out[:,1],np.array(labels)[:,1])
    print('Correct Key in '+str(AccK)+'% - Precision: '+
          str(PK)+'% Recall: '+str(RK)+'%  |  TP,TN,FP,FN: '+str(KA))

    AccO,PO,RO,OA = getAccPR(out[:,2],np.array(labels)[:,2])
    print('Correct Orb in '+str(AccO)+'% - Precision: '+
          str(PO)+'% Recall: '+str(RO)+'%  |  TP,TN,FP,FN: '+str(OA))
    
def getStats(out, labels):

    AccND,PND,RND,NDA = getAccPR((out[:,0]==0).astype(int),(labels[:,0]==0).astype(int))
    
    AccLD,PLD,RLD,LDA = getAccPR((out[:,0]==1).astype(int),(labels[:,0]==1).astype(int))

    AccGD,PGD,RGD,GDA = getAccPR((out[:,0]==2).astype(int),(labels[:,0]==2).astype(int))

    AccKD,PKD,RKD,KDA = getAccPR((out[:,0]==3).astype(int),(labels[:,0]==3).astype(int))

    AccOD,POD,ROD,ODA = getAccPR((out[:,0]==4).astype(int),(labels[:,0]==4).astype(int))

    AccK,PK,RK,KA = getAccPR(out[:,1],np.array(labels)[:,1])

    AccO,PO,RO,OA = getAccPR(out[:,2],np.array(labels)[:,2])
    
    precisions = np.nan_to_num(np.array([PND, PLD, PGD, PKD, POD, PK, PO]))
    recalls = np.nan_to_num(np.array([RND, RLD, RGD, RKD, ROD, RK, RO]))
    f1scores = np.nan_to_num(np.round(2*(precisions*recalls/(precisions+recalls)),2))
    
    stats = {'Accuracy': np.mean(out == labels)*100,
            'Accuracies': np.array([AccND, AccLD, AccGD, AccKD, AccOD, AccK, AccO]),
            'Precisions': precisions,
            'Recalls': recalls,
            'F1Scores': f1scores}
    return stats


### Learning Concepts
#### Single Concepts

In [18]:
def learnSingleConcept(examples, pattern_complexity,normalize = False):
    neuronActs = np.sum(examples,axis=0)
    strongest = neuronActs.argsort()[-pattern_complexity:][::-1]
    strongest_N = []
    strengths = []
    for s in strongest:
        #print(str(s) + '        ' + str(int(neuronActs[s])))
        strongest_N.append(s)
        strengths.append(int(neuronActs[s]))#/examples.shape[0])
    if normalize:
        strengths = strengths/np.sum(strengths)
    else:
        strengths = np.array(strengths)/examples.shape[0]
    return strongest_N, strengths

def getSingleActsLogic(neurons, strengths, data, class_TH):
    #strengths = strengths/np.max(strengths)
    patternOn = np.zeros(data.shape[0])
    for i,n in enumerate(neurons):
        n1_act = (data[:,int(n)] > 0.5)
        n_act = np.array(n1_act, dtype=int) * strengths[i]
        #print(n_act)
        patternOn = patternOn + n_act
    '''print('pattern on: ' + str(patternOn))
    print('Max: ' + str(np.max(patternOn))[:4])
    print('Min: ' + str(np.min(patternOn)))
    print('Mean: ' + str(np.mean(patternOn)))
    print('Median: ' + str(np.median(patternOn)))'''
    #plt.hist(patternOn)
    patternOn = (patternOn >= class_TH)#np.array(neuronCombs).shape[0]-1)
    return patternOn

def LearnAllSingleConcepts(numExp, pattern_complexity, encoding, to_learn, concept_def):
    allConcepts = []
        
    for i, concept in enumerate(to_learn):
        learnExamples, testExamples = getConceptSubset(np.where(concept_def[i])[0], numExp)

        examples = encoding[learnExamples]
        labels_test = concept_def[i]

        combs, weights = learnSingleConcept(examples, pattern_complexity=pattern_complexity,normalize=True)
        
        true_exp = np.where(labels_test==True)[0]
        false_exp = np.where(labels_test==False)[0]
        np.random.shuffle(false_exp)
        balanced_exp = np.append(true_exp,false_exp[:true_exp.shape[0]])

        cpr_train = getCPR(combs, weights, examples, np.ones(examples.shape[0]), np.linspace(0,1,21),pattern='single')
        cpr_test = getCPR(combs, weights, encoding[balanced_exp], labels_test[balanced_exp], np.linspace(0,1,21),pattern='single')
        
        ConceptStats = {'Concept':concept,
                     'Examples': learnExamples,
                   'TestExamples': testExamples,
                     'ExamplesEnc': examples,
                     'Combinations': combs,
                     'Weights': weights,
                     'CPR_Train': cpr_train,
                     'CPR_Test': cpr_test}

        allConcepts.append(ConceptStats)
    return allConcepts

## FCM
### Define Parameters

In [19]:
to_learn = ['Level Door', 'Green Door', 'Key Door', 'Other Door', 'Key', 'Orb']#'No Door', 
concept_def = [y[:,0]==1, y[:,0]==2, y[:,0]==3, y[:,0]==4, y[:,1]==1, y[:,2]==1]#y[:,0]==0, 
numExp_default = 6
pattern_complexity_default = 12

### Test

In [20]:
allConceptsSingle = LearnAllSingleConcepts(numExp_default, pattern_complexity_default, encBYOL, to_learn, concept_def)

## Find Best TH for all Conditions

In [8]:
numRuns = 100
bestTHStatsAEBCR_TH = pd.DataFrame(columns = {'Concept','Encoding','Pattern Complexity', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall'})

for pattern_complexity in [pattern_complexity_default]:
    for numExp in [numExp]:
        for i in range(numRuns):
            allConcepts = LearnAllSingleConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            allConceptsAE = LearnAllSingleConcepts(numExp, pattern_complexity, encAE, to_learn, concept_def)
            allConceptsBYOL = LearnAllSingleConcepts(numExp, pattern_complexity, encBYOL, to_learn, concept_def)
            allConceptsC = LearnAllSingleConcepts(numExp, pattern_complexity, encC, to_learn, concept_def)
            allConceptsR = LearnAllSingleConcepts(numExp, pattern_complexity, encR, to_learn, concept_def)
            for c in allConcepts:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAEBCR_TH = bestTHStatsAEBCR_TH.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)
                
            for c in allConceptsAE:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAEBCR_TH = bestTHStatsAEBCR_TH.append({'Concept': c['Concept'],'Encoding':'Autoencoder','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)

            for c in allConceptsBYOL:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAEBCR_TH = bestTHStatsAEBCR_TH.append({'Concept': c['Concept'],'Encoding':'BYOL','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)
                
            for c in allConceptsC:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAEBCR_TH = bestTHStatsAEBCR_TH.append({'Concept': c['Concept'],'Encoding':'Classifier','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)
            for c in allConceptsR:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAEBCR_TH = bestTHStatsAEBCR_TH.append({'Concept': c['Concept'],'Encoding':'Random','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)


In [9]:
bestTHStatsAEBCR_TH.to_csv(figurePath+'bestTHStatsAEBCR_TH.csv')

In [10]:
import pandas as pd
bestTHStatsAEBCR_TH = pd.read_csv(figurePath + 'bestTHStatsAEBCR_TH.csv')

In [11]:
for e in bestTHStatsAEBCR_TH['Encoding'].unique():
    data = bestTHStatsAEBCR_TH[bestTHStatsAEBCR_TH['Encoding']==e]
    print(e)
    bestTH = 0
    bestTHInd = 0
    for th in np.linspace(0,1,21):
        dataTH = data[data['TH']==np.round(th,2)]
        if np.mean(dataTH['Accuracy']) > bestTH:
            bestTH = np.mean(dataTH['Accuracy'])
            bestTHInd = th
            #print("updated to th = " + str(bestTHInd))
    print("Best TH=" + str(np.round(bestTHInd,2)) + " with mean Accuracy= " + str(bestTH))


Agent
Best TH=0.4 with mean Accuracy= 65.246
Autoencoder
Best TH=0.5 with mean Accuracy= 55.87466666666667
BYOL
Best TH=0.6 with mean Accuracy= 60.03433333333333
Classifier
Best TH=0.55 with mean Accuracy= 66.30566666666667
Random
Best TH=0.9 with mean Accuracy= 50.92466666666666


### FCM on 5 Network Activations
Comparing FCM on activations of agent, BYOL, autoencoder, classifier and random.

Using best TH for each condition.


In [20]:
numRuns = 100
bestIndvTHStatsAEBCR = pd.DataFrame(columns = {'Concept','Encoding', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})

for pattern_complexity in [pattern_complexity_default]:
    for numExp in [numExp_default]:
        for i in range(numRuns):
            print(i)
            allConcepts = LearnAllSingleConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            allConceptsAE = LearnAllSingleConcepts(numExp, pattern_complexity, encAE, to_learn, concept_def)
            allConceptsBYOL = LearnAllSingleConcepts(numExp, pattern_complexity, encBYOL, to_learn, concept_def)
            allConceptsC = LearnAllSingleConcepts(numExp, pattern_complexity, encC, to_learn, concept_def)
            allConceptsR = LearnAllSingleConcepts(numExp, pattern_complexity, encR, to_learn, concept_def)
            for c in allConcepts:
                TH20 = c['CPR_Test'][8]
                bestIndvTHStatsAEBCR = bestIndvTHStatsAEBCR.append({'Concept': c['Concept'],
                                                                            'Encoding':'Agent',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.4,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
            for c in allConceptsAE:
                TH20 = c['CPR_Test'][10]
                bestIndvTHStatsAEBCR = bestIndvTHStatsAEBCR.append({'Concept': c['Concept'],
                                                                            'Encoding':'Autoencoder',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.5,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)

            for c in allConceptsBYOL:
                TH20 = c['CPR_Test'][12]
                bestIndvTHStatsAEBCR = bestIndvTHStatsAEBCR.append({'Concept': c['Concept'],
                                                                            'Encoding':'BYOL',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.6,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
               
            for c in allConceptsC:
                TH20 = c['CPR_Test'][11]
                bestIndvTHStatsAEBCR = bestIndvTHStatsAEBCR.append({'Concept': c['Concept'],
                                                                            'Encoding':'Classifier',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.55,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
           
            for c in allConceptsR:
                TH20 = c['CPR_Test'][18]
                bestIndvTHStatsAEBCR = bestIndvTHStatsAEBCR.append({'Concept': c['Concept'],
                                                                            'Encoding':'Random',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.9,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
           


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [21]:
bestIndvTHStatsAEBCR.to_csv(figurePath+'bestIndvTHStatsAEBCR.csv')

### FCM on all Activations for Different Numbers of Examples

In [27]:
numRuns = 100
bestTHStatsAEBCR_numExp = pd.DataFrame(columns = {'Concept','Encoding','PC', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})

to_learnD = [to_learn[0]]
concept_defD = [concept_def[0]]
for pattern_complexity in [pattern_complexity_default]:
    for numExp in [1,2,3,4,5,6,12,24,48]:
        for i in range(numRuns):
            print(i)
            allConcepts = LearnAllSingleConcepts(numExp, pattern_complexity, encInt, to_learnD, concept_defD)
            allConceptsAE = LearnAllSingleConcepts(numExp, pattern_complexity, encAE, to_learnD, concept_defD)
            allConceptsBYOL = LearnAllSingleConcepts(numExp, pattern_complexity, encBYOL, to_learnD, concept_defD)
            allConceptsC = LearnAllSingleConcepts(numExp, pattern_complexity, encC, to_learnD, concept_defD)
            allConceptsR = LearnAllSingleConcepts(numExp, pattern_complexity, encR, to_learnD, concept_defD)
            for c in allConcepts:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][7]
                    bestTHStatsAEBCR_numExp = bestTHStatsAEBCR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'Agent',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.35,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
            
            for c in allConceptsAE:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][8]
                    bestTHStatsAEBCR_numExp = bestTHStatsAEBCR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'Autoencoder',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.4,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)

            for c in allConceptsBYOL:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][11]
                    bestTHStatsAEBCR_numExp = bestTHStatsAEBCR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'BYOL',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.55,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
            for c in allConceptsC:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][11]
                    bestTHStatsAEBCR_numExp = bestTHStatsAEBCR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'Classifier',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.55,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
            for c in allConceptsR:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][15]
                    bestTHStatsAEBCR_numExp = bestTHStatsAEBCR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'Random',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.75,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46

In [28]:
bestTHStatsAEBCR_numExp.to_csv(figurePath+'bestTHStatsAEBCR_numExp_LDoor.csv')

### FCM on all Activations for Different Pattern Complexities

In [21]:
numRuns = 10
bestTHStatsAEBCR_PCComp_AllC = pd.DataFrame(columns = {'Concept','Encoding','PC', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})

for pattern_complexity in [1,2,3,4,5,6,12,18,48]:
    for numExp in [numExp_default]:
        for i in range(numRuns):
            print(str(pattern_complexity) + ' - ' + str(i))
            allConcepts = LearnAllSingleConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            allConceptsAE = LearnAllSingleConcepts(numExp, pattern_complexity, encAE, to_learn, concept_def)
            allConceptsBYOL = LearnAllSingleConcepts(numExp, pattern_complexity, encBYOL, to_learn, concept_def)
            allConceptsC = LearnAllSingleConcepts(numExp, pattern_complexity, encC, to_learn, concept_def)
            allConceptsR = LearnAllSingleConcepts(numExp, pattern_complexity, encR, to_learn, concept_def)
            for c in allConcepts:
                TH20 = c['CPR_Test'][7]
                bestTHStatsAEBCR_PCComp_AllC = bestTHStatsAEBCR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'Agent',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.35,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
            for c in allConceptsAE:
                TH20 = c['CPR_Test'][8]
                bestTHStatsAEBCR_PCComp_AllC = bestTHStatsAEBCR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'Autoencoder',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.4,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)

            for c in allConceptsBYOL:
                TH20 = c['CPR_Test'][11]
                bestTHStatsAEBCR_PCComp_AllC = bestTHStatsAEBCR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'BYOL',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.55,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
            for c in allConceptsC:
                TH20 = c['CPR_Test'][11]
                bestTHStatsAEBCR_PCComp_AllC = bestTHStatsAEBCR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'Classifier',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.55,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
            for c in allConceptsR:
                TH20 = c['CPR_Test'][15]
                bestTHStatsAEBCR_PCComp_AllC = bestTHStatsAEBCR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'Random',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.75,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
    # intermediate saves in case of memory issues:
    # bestTHStatsAEBCR_PCComp_AllC.to_csv(figurePath+'bestTHStatsAEBCR_PCComp_AllC'+str(pattern_complexity)+'.csv')
    

1 - 0
1 - 1
1 - 2
1 - 3
1 - 4
1 - 5
1 - 6
1 - 7
1 - 8
1 - 9
2 - 0
2 - 1
2 - 2
2 - 3
2 - 4
2 - 5
2 - 6
2 - 7
2 - 8
2 - 9
3 - 0
3 - 1
3 - 2
3 - 3
3 - 4
3 - 5
3 - 6
3 - 7
3 - 8
3 - 9
4 - 0
4 - 1
4 - 2
4 - 3
4 - 4
4 - 5
4 - 6
4 - 7
4 - 8
4 - 9
5 - 0
5 - 1
5 - 2
5 - 3
5 - 4
5 - 5
5 - 6
5 - 7
5 - 8
5 - 9
6 - 0
6 - 1
6 - 2
6 - 3
6 - 4
6 - 5
6 - 6
6 - 7
6 - 8
6 - 9
12 - 0
12 - 1
12 - 2
12 - 3
12 - 4
12 - 5
12 - 6
12 - 7
12 - 8
12 - 9
18 - 0
18 - 1
18 - 2
18 - 3
18 - 4
18 - 5
18 - 6
18 - 7
18 - 8
18 - 9
48 - 0
48 - 1
48 - 2
48 - 3
48 - 4
48 - 5
48 - 6
48 - 7
48 - 8
48 - 9


In [22]:
bestTHStatsAEBCR_PCComp_AllC.to_csv(figurePath+'bestIndvTHStatsAEBCR_PCComp_AllC.csv')

# TODO: Run for 100 tests with long run data

### FCM on Different Agent Reward Types

In [ ]:
numRuns = 100
bestTHStatsA_TH20 = pd.DataFrame(columns = {'Concept','Encoding','Pattern Complexity', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})
for pattern_complexity in [pattern_complexity_default]:
    for numExp in [numExp_default]:
        for i in range(numRuns):
            print(i)
            allConceptsE = LearnAllSingleConcepts(numExp, pattern_complexity, encExt, to_learn, concept_def)
            allConceptsEI = LearnAllSingleConcepts(numExp, pattern_complexity, encExtInt, to_learn, concept_def)
            allConceptsI = LearnAllSingleConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            
            for c in allConceptsE:
                TH20 = c['CPR_Test'][4]
                bestTHStatsA_TH20 = bestTHStatsA_TH20.append({'Concept': c['Concept'],
                                                                            'Encoding':'External',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                     
            for c in allConceptsEI:
                TH20 = c['CPR_Test'][4]
                bestTHStatsA_TH20 = bestTHStatsA_TH20.append({'Concept': c['Concept'],
                                                                            'Encoding':'Ext+Int',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)

            for c in allConceptsI:
                TH20 = c['CPR_Test'][4]
                bestTHStatsA_TH20 = bestTHStatsA_TH20.append({'Concept': c['Concept'],
                                                                            'Encoding':'Internal',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
                


In [ ]:
bestTHStatsA_TH20.to_csv(figurePath+'bestTHStatsAgentsNormx2_TH20.csv')